### Generate 5 recommendation plans for each user in the dataset

In [ ]:
alpha = 0.25
theta = 1.5

optional = pd.DataFrame(columns=['name','gender','height_cm','weight_kg','age','food ID','Descrip','last eat timestamp','frequency_day'])

user_recommendation = pd.DataFrame(columns=['name','Breakfast','Lunch','Dinner','Energy_kcal','Protein_g', 'Fat_g', 'Carb_g', 'Sugar_g', 'weight'])

for loop in range(49):
  user_record = users.iloc[loop*RECORD_NO: (loop+1)*RECORD_NO].reset_index()

  #get user prefernece table: weight table
  weight_record = getWeightRecord(loop, theta, optional)

  # user info
  user_name = user_record.at[0, 'name']
  user_age = user_record.at[0, 'age']
  user_gender = user_record.at[0, 'gender']
  user_height = user_record.at[0, 'height_cm']
  user_weight = user_record.at[0, 'weight_kg']

  #calculate user BMR
  user_BMR = getBMR(user_height, user_weight, user_age, user_gender)

  #generate 1000 random breakfast plan for user
  initial_b = getBreakfast(1000)
  filter_b = filterBreakfast(initial_b, user_BMR)

  #generate 1000 random lunch plan for user
  initial_l = getLunch(1000)
  filter_l = filterLunch(initial_l, user_BMR)

  #generate 1000 random dinner plan for user
  initial_d = getLunch(1000)
  filter_d = filterDinner(initial_d, user_BMR)

  #generate 2000 random menu set for user
  if len(filter_b) == 0 or len(filter_l) == 0 or len(filter_d) == 0:
    empty = {'name': [user_name], 'Breakfast':['No recommendation'], 'Lunch':['No recommendation'], 'Dinner':['No recommendation'], 'Energy_kcal':[0], 'Protein_g':[0], 'Fat_g':[0], 'Carb_g':[0], 'Sugar_g':[0], 'weight':[0]}
    user_recommendation = user_recommendation.append(pd.DataFrame(empty))
    continue
  initial_m = getSet(filter_b, filter_l, filter_d, 2000)
  filter_m = filterSet(initial_m, user_weight, user_BMR, alpha)

  recommend = calculateW4Set(filter_m, weight_record)
  final_recommend = finalSets(recommend).sort_values('weight', ascending=False).iloc[0:5]

  final_recommend.insert(0, 'name', [user_name]*len(final_recommend))

  user_recommendation = user_recommendation.append(final_recommend)


In [ ]:
user_recommendation = user_recommendation.reset_index()
user_recommendation

,index,name,Breakfast,Lunch,Dinner,Energy_kcal,Protein_g,Fat_g,Carb_g,Sugar_g,weight
0,499,Laura,"Yogurt, fruit, low fat, 10 grams protein per 8...","Turkey from whole, enhanced, light meat, meat ...","Lamb, New Zealand, imported, leg chop/steak, b...",1268.0,69.37,38.0,172.67,51.76,4.208907
1,106,Laura,"Yogurt, fruit, low fat, 10 grams protein per 8...","Turkey from whole, enhanced, light meat, meat ...","HORMEL ALWAYS TENDER, Pork Tenderloin, Pepperc...",1245.0,62.32,20.31,217.31,35.88,3.844284
2,434,Laura,"Yogurt, fruit, lowfat, with low calorie sweete...","Turkey from whole, enhanced, light meat, meat ...","Beef, Australian, grass-fed, loin, top sirloin...",1233.0,64.72,22.98,201.85,35.59,3.6752
3,57,Laura,"Yogurt, fruit, lowfat, with low calorie sweete...","Salami, cooked, turkey, Pimento, canned, Bread...","OSCAR MAYER, Ham (water added, honey), Pineapp...",1327.0,63.35,27.07,216.33,52.83,3.142695
4,203,Laura,"Yogurt, fruit, lowfat, with low calorie sweete...","Salami, cooked, turkey, Pimento, canned, Bread...","Milk, low sodium, fluid, Cauliflower, cooked, ...",1310.0,63.15,23.2,226.72,40.62,3.001554
...,...,...,...,...,...,...,...,...,...,...,...
184,291,Roger,"Milk, low sodium, fluid, Cereals, WHEATENA, co...","Veal, loin, chop, separable lean and fat, cook...","Milk, lowfat, fluid, 1% milkfat, with added no...",1310.0,69.48,27.62,201.05,44.19,4.19297
185,298,Roger,"Cheese, cottage, nonfat, uncreamed, dry, large...","Lamb, New Zealand, imported, loin chop, separa...","Ham, minced, Macaroni, cooked, unenriched, Kal...",1419.0,68.94,58.99,159.16,24.44,4.189185
186,378,Roger,"Cheese, cottage, nonfat, uncreamed, dry, large...","Beef, chuck, under blade center steak, boneles...","Crustaceans, shrimp, untreated, raw, Beans, sn...",1369.0,68.96,42.1,191.78,42.17,3.342718
187,0,Eleanor,No recommendation,No recommendation,No recommendation,0,0,0,0,0,0


In [ ]:
valid = user_recommendation.loc[user_recommendation['Breakfast'] != 'No recommendation']
valid

,index,name,Breakfast,Lunch,Dinner,Energy_kcal,Protein_g,Fat_g,Carb_g,Sugar_g,weight
0,499,Laura,"Yogurt, fruit, low fat, 10 grams protein per 8...","Turkey from whole, enhanced, light meat, meat ...","Lamb, New Zealand, imported, leg chop/steak, b...",1268.0,69.37,38.0,172.67,51.76,4.208907
1,106,Laura,"Yogurt, fruit, low fat, 10 grams protein per 8...","Turkey from whole, enhanced, light meat, meat ...","HORMEL ALWAYS TENDER, Pork Tenderloin, Pepperc...",1245.0,62.32,20.31,217.31,35.88,3.844284
2,434,Laura,"Yogurt, fruit, lowfat, with low calorie sweete...","Turkey from whole, enhanced, light meat, meat ...","Beef, Australian, grass-fed, loin, top sirloin...",1233.0,64.72,22.98,201.85,35.59,3.6752
3,57,Laura,"Yogurt, fruit, lowfat, with low calorie sweete...","Salami, cooked, turkey, Pimento, canned, Bread...","OSCAR MAYER, Ham (water added, honey), Pineapp...",1327.0,63.35,27.07,216.33,52.83,3.142695
4,203,Laura,"Yogurt, fruit, lowfat, with low calorie sweete...","Salami, cooked, turkey, Pimento, canned, Bread...","Milk, low sodium, fluid, Cauliflower, cooked, ...",1310.0,63.15,23.2,226.72,40.62,3.001554
...,...,...,...,...,...,...,...,...,...,...,...
182,426,Roger,"Cheese, cottage, nonfat, uncreamed, dry, large...","Beef, plate steak, boneless, outside skirt, se...","Pork, cured, ham with natural juices, slice, b...",1373.0,76.06,33.69,204.58,54.96,5.697592
183,379,Roger,"Yogurt, fruit, lowfat, with low calorie sweete...","Veal, variety meats and by-products, tongue, r...","Pork, cured, ham with natural juices, slice, b...",1437.0,87.12,45.68,181.95,52.64,5.163885
184,291,Roger,"Milk, low sodium, fluid, Cereals, WHEATENA, co...","Veal, loin, chop, separable lean and fat, cook...","Milk, lowfat, fluid, 1% milkfat, with added no...",1310.0,69.48,27.62,201.05,44.19,4.19297
185,298,Roger,"Cheese, cottage, nonfat, uncreamed, dry, large...","Lamb, New Zealand, imported, loin chop, separa...","Ham, minced, Macaroni, cooked, unenriched, Kal...",1419.0,68.94,58.99,159.16,24.44,4.189185


We can see that most users get recommendation from our system. The reason why some of them do not get recommendation is because our system filters all breakfast/lunch/dinner out because they do not meet nutritional requirement (e.g., maximum daily sugar, fat income).
\
Another reason would be that our user dataset is randomly genederted, which may contain many data that is not realistic. This would make system hard to generate personal meal correctly.
